# First, let's read the celebrity name CSVs 

In [2]:
# Let us install the library we are going to use to download images:
! pip install simple_image_download
! pip install scikit-image

In [3]:
import pandas as pd
import boto3
import json
import io
import cv2
import skimage

import numpy as np
from IPython.display import Image as IImage
from skimage.exposure import is_low_contrast

from PIL import Image, ImageDraw, ExifTags, ImageColor
from matplotlib.pyplot import imshow


In [4]:
!ls

chinese_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/chinese_celebs.csv')

western_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/western_celebs.csv')


chinese_celebs.csv		  simple_images
ForbesCelebrityComparisons.ipynb  western_celebs.csv


In [5]:
chinese_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Fan Bingbing,300 million yuan,Actress,China
1,2,Lu Han,210 million yuan,Actor and Singer,China
2,3,Yang Mi,200 million yuan,Actress,China
3,4,Zhao Liying,190 million yuan,Actress,China
4,5,Yang Yang,240 million yuan,Actor,China
...,...,...,...,...,...
73,75,Xie Na,40 million yuan,TV Host and Actress,China
74,76,Hawick Lau,75 million yuan,Actor,China
75,78,Huang Xuan,50 million yuan,Actor,China
76,79,Ma Sichun,40 million yuan,Actress,China


In [6]:
western_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Diddy,$130 million,Musician,U.S.
1,2,Beyoncé,$105 million,Musician,U.S.
2,3,J.K. Rowling,$95 million,Author,U.K.
3,4,Drake,$94 million,Musician,Canada
4,6,The Weeknd,$92 million,Musician,Canada
...,...,...,...,...,...
75,95,Chance The Rapper,$33 million,Musician,U.S.
76,95,Katy Perry,$33 million,Musician,U.S.
77,97,Carmelo Anthony,$32.6 million,Athlete,U.S.
78,98,Jason Aldean,$32.5 million,Musician,U.S.


In [7]:
from simple_image_download import simple_image_download as simp 
chinese_celebrity_names = chinese_celebs['Name'][0:50]

response = simp.simple_image_download

for name in chinese_celebrity_names:
 response().download(name +' face' , 50)

HTTPSConnectionPool(host='www.chloetan.com', port=443): Max retries exceeded with url: /wp-content/uploads/2016/10/andy_0.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f0aeee06710>, 'Connection to www.chloetan.com timed out. (connect timeout=None)'))


KeyboardInterrupt: 

In [ ]:
chinese_celeb_bucket = 'chinese-celebs'
!aws s3 mb s3://$chinese_celeb_bucket

# This is the AWS CLI command to create an S3 bucket. 
# The mb stands for "make bucket," and it's used to create a new S3 bucket.
# 's3://asian-faces-bucket' indicates that we're making our bucket in s3, and the part after the '//' is the name of the bucket


In [ ]:
foldername = 'simple_images'
!ls $foldername 
!aws s3 mv ./$foldername s3://$chinese_celeb_bucket/ --recursive # move the contents of the folder into the s3 bucket


In [ ]:
client=boto3.client('rekognition')
import subprocess
import re
import s3fs 

s3 = boto3.client('s3')

response = s3.list_objects_v2(Bucket=chinese_celeb_bucket)

# Iterate through the objects in the bucket
for obj in response.get('Contents', []):
    # Get the object key (file path)
    object_key = obj['Key']
    # Download the object to a local file (if needed)
    # s3.download_file(bucket_name, object_key, 'local_file_path')

    # Apply your Python function to the object
    # Example: print the object key
    print("Processing file:", object_key)

In [ ]:
def should_remove_photo(photo, bucket):        
    response = client.detect_labels(Image={'S3Object':{'Bucket': bucket,
                                                   'Name': photo}}, MaxLabels=10)
    #perform image check
    image = Image.open(photo)
    imgWidth, imgHeight = image.size 
    faceDetail = response['FaceDetails'][0]
    pitch = faceDetail['Pose']['Pitch']
    yaw = faceDetail['Pose']['Yaw']
    sunglasses = faceDetail['Sunglasses']['Value']
    box = faceDetail['BoundingBox']
    width = imgWidth * box['Width']
    height = imgHeight * box['Height']
    eyesOpen = faceDetail['EyesOpen']['Value']
    emotion = faceDetail['Emotions'][0]['Type']
    emotionConfidence = faceDetail['Emotions'][0]['Confidence']
    mouthOpen = faceDetail['MouthOpen']['Value']
    image_results = ""
        
    if width < 50 or height < 50:
        return True
    if abs(yaw) > 45:
        return True
    if pitch > 45 or pitch < -30:
        return True
    if not eyesOpen or sunglasses or mouthOpen:
        return True
    return False
 
def low_contrast_check (photo):
    image = cv2.imread(photo)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return is_low_contrast(gray, fraction_threshold=0.35)

%matplotlib inline


In [ ]:
import boto3
from PIL import Image
from io import BytesIO

# Initialize the S3 client
s3 = boto3.client('s3')

bucket_name = 'chinese-celebs'

# List objects in the bucket
response = s3.list_objects_v2(Bucket=bucket)

# Initialize Rekognition client
rekognition_client = boto3.client('rekognition')

# Function to remove photo based on your criteria
def should_remove_photo(photo, bucket):
    # Perform label detection using Rekognition
    response = rekognition_client.detect_faces(Image={'S3Object': {'Bucket': bucket_name, 'Name': photo}})
    print(response)
    # Perform image checks
    if response['FaceDetails'] == []:
        return True
    
    image = Image.open(BytesIO(s3.get_object(Bucket=bucket, Key=photo)['Body'].read()))
    
    imgWidth, imgHeight = image.size

    face_details = response['FaceDetails'][0] if 'FaceDetails' in response and response['FaceDetails'] else None
    pitch = face_details['Pose']['Pitch'] if 'Pose' in face_details and 'Pitch' in face_details['Pose'] else None
    yaw = face_details['Pose']['Yaw'] if 'Pose' in face_details and 'Yaw' in face_details['Pose'] else None
    sunglasses = face_details.get('Sunglasses', {}).get('Value', None)
    bounding_box = face_details.get('BoundingBox', {})
    width = imgWidth * bounding_box.get('Width', 0)
    height = imgHeight * bounding_box.get('Height', 0)
    eyes_open = face_details.get('EyesOpen', {}).get('Value', None)
    
    # Check if Emotions array is present and non-empty
    emotions = face_details.get('Emotions', [])
    emotion = emotions[0]['Type'] if emotions else None
    emotion_confidence = emotions[0]['Confidence'] if emotions else None
    
    mouth_open = face_details.get('MouthOpen', {}).get('Value', None)

    
    if width < 50 or height < 50:
        print('Too small')
        return True
    if abs(yaw) > 45:
        print('yaw too much')
        return True
    if pitch > 45 or pitch < -30:
        print('pitch too severe')
        return True
    if not eyes_open or sunglasses or mouthOpen:
        print('eye, sunglasses, or mouth')
        return True
    return False

# Iterate through the objects in the bucket
for obj in response.get('Contents', []):
    # Get the object key (file path)
    object_key = obj['Key']
    print(object_key)

    # Apply your function to the object
    if should_remove_photo(object_key, bucket_name):
        print("Removing photo:", object_key)

        # Remove the file from the bucket
        s3.delete_object(Bucket=bucket_name, Key=object_key)
        
        print("Photo removed.")
    else:
        print("Keeping photo:", object_key)
